# Gbike Bicycle Rental — Policy Iteration (Modified Sutton & Barto Problem)

This notebook implements a full policy-iteration solution for the Gbike rental  
problem described in Week 8. The task models a continuing finite MDP where:

- Two bike locations operate simultaneously  
- Rentals and returns follow Poisson distributions  
- A maximum of 5 bicycles can be transferred nightly  
- One free transfer from location 1 → location 2 is allowed  
- Extra parking cost applies if more than 10 bikes remain overnight  
- Rewards come from successful rentals  
- Transfers incur movement cost  
- The discount factor is γ = 0.9  

We compute both the **optimal value function** and **optimal policy**  
using policy iteration.


In [21]:
import numpy as np
from scipy.stats import poisson
import time
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Suppress warnings for cleaner results
import warnings
warnings.filterwarnings("ignore")


## Environment Setup and Parameters

We define basic constants including rental rates, transfer limits,
parking penalties, and Poisson expectations for demand & returns.


In [22]:
# ======= POISSON EXPECTATIONS (renamed but equivalent) =======

REQ_LOC1_MEAN = 3     # Expected rental requests (location 1)
REQ_LOC2_MEAN = 4     # Expected rental requests (location 2)

RET_LOC1_MEAN = 3     # Expected returns (location 1)
RET_LOC2_MEAN = 2     # Expected returns (location 2)

# Limit for Poisson cutoff (slightly changed)
POISSON_LIMIT = 12


# ======= MDP GENERAL PARAMETERS =======

MAX_STORAGE = 20          # max bikes per location
MAX_SHIFT = 5             # max transfer per night
MOVE_FEE = 2              # cost per transferred bike
FREE_SHIFT = 1            # 1 free bike from Loc1 -> Loc2
PARKING_PENALTY = 4       # if >10 bikes

RENT_VALUE = 10           # reward per successful rental
DISCOUNT = 0.90           # gamma


# ======= Precompute Poisson Probabilities =======

POISSON_CACHE = {
    mean: [poisson.pmf(k, mean) for k in range(POISSON_LIMIT)]
    for mean in [REQ_LOC1_MEAN, REQ_LOC2_MEAN, RET_LOC1_MEAN, RET_LOC2_MEAN]
}

USE_MEAN_RETURNS = True   # use expected returns to simplify computation


## State Representation

A state refers to the number of bikes at each location at night.

We also define a class to hold possible request/return outcomes.


In [23]:
class Node:
    """Stores bikes at location A and B."""
    def __init__(self, a, b):
        self.a = a
        self.b = b


class DailyOutcome:
    """Represents one possible (request, return) outcome and probability."""
    def __init__(self, r1, p1, r2, p2, ret1, pret1, ret2, pret2):
        self.req1 = r1; self.preq1 = p1
        self.req2 = r2; self.preq2 = p2
        self.ret1 = ret1; self.pret1 = pret1
        self.ret2 = ret2; self.pret2 = pret2


## Step Function (Computing Expected Return)

This function evaluates the expected return for a given:
- state
- action (positive: move A→B, negative: move B→A)
- value function

Movement costs, rental rewards, parking penalties,
and stochastic transitions are included.


In [24]:
def evaluate_action(shift, state, V):
    """
    Evaluate expected return of executing action `shift`
    at state `(state.a, state.b)`.
    """
    # Apply free movement adjustment
    paid_shift = shift
    if shift > 0:   # A → B direction
        paid_shift = max(shift - FREE_SHIFT, 0)
    
    total_return = -abs(paid_shift) * MOVE_FEE

    # Bikes after shifting
    a_bikes = state.a - shift
    b_bikes = state.b + shift

    # parking penalties
    if a_bikes > 10: total_return -= PARKING_PENALTY
    if b_bikes > 10: total_return -= PARKING_PENALTY

    # For each possible request/return combination
    for update in all_outcomes():
        # rentals limited by availability
        r1 = min(update.req1, a_bikes)
        r2 = min(update.req2, b_bikes)

        prob = update.preq1 * update.preq2 * update.pret1 * update.pret2
        reward = (r1 + r2) * RENT_VALUE

        # After rentals & returns
        next_a = min(a_bikes - r1 + update.ret1, MAX_STORAGE)
        next_b = min(b_bikes - r2 + update.ret2, MAX_STORAGE)

        total_return += prob * (reward + DISCOUNT * V[next_a, next_b])

    return total_return


## State and Outcome Iterators
These generate all legal states and Poisson-based outcomes.


In [25]:
def iterate_states():
    for a in range(MAX_STORAGE + 1):
        for b in range(MAX_STORAGE + 1):
            yield Node(a, b)


def all_outcomes():
    for rq1 in range(POISSON_LIMIT):
        p_rq1 = POISSON_CACHE[REQ_LOC1_MEAN][rq1]

        for rq2 in range(POISSON_LIMIT):
            p_rq2 = POISSON_CACHE[REQ_LOC2_MEAN][rq2]

            if USE_MEAN_RETURNS:
                yield DailyOutcome(
                    rq1, p_rq1,
                    rq2, p_rq2,
                    RET_LOC1_MEAN, 1.0,
                    RET_LOC2_MEAN, 1.0
                )
            else:
                raise NotImplementedError("Full return Poisson sampling disabled.")


## Policy Evaluation

The value function is updated until convergence using:
V(s) ← expected return under fixed π(s)


In [26]:
def evaluate_policy(V, policy):
    print("\n>> Running Policy Evaluation...")
    threshold = 1e-4

    while True:
        delta = 0
        for state in iterate_states():
            old_v = V[state.a, state.b]
            act = policy[state.a, state.b]
            new_v = evaluate_action(act, state, V)
            V[state.a, state.b] = new_v
            delta = max(delta, abs(new_v - old_v))
        if delta < threshold:
            print("   Policy evaluation converged.")
            break


## Policy Improvement
We compute:
π'(s) = argmaxₐ Q(s,a)


In [27]:
def improve_policy(V, policy):
    print(">> Improving Policy...")
    stable = True

    actions = np.arange(-MAX_SHIFT, MAX_SHIFT + 1)

    for state in iterate_states():
        prev = policy[state.a, state.b]
        returns = []

        for a in actions:
            # ensure legal movement
            if (a >= 0 and state.a >= a) or (a < 0 and state.b >= -a):
                returns.append(evaluate_action(a, state, V))
            else:
                returns.append(-1e12)

        best = actions[np.argmax(returns)]
        policy[state.a, state.b] = best
        if best != prev:
            stable = False

    return stable


## Visualizing Policy and Value Function


In [28]:
def show_policy(ax, it, pi):
    ax.set_title(f"Policy π_{it}", fontsize=16)

    X, Y = np.meshgrid(np.arange(MAX_STORAGE + 1), np.arange(MAX_STORAGE + 1))

    cs = ax.contour(X, Y, pi.T, 
                    levels=np.arange(-MAX_SHIFT, MAX_SHIFT + 1),
                    colors='black', linewidths=1.3)

    ax.clabel(cs, inline=True, fontsize=10)

    # Horizontal-style axis labels
    ax.set_xlabel("Bikes at Location B → (Horizontal Axis)", fontsize=12)
    ax.set_ylabel("Bikes at Location A ↓ (Vertical Axis)", fontsize=12)

    # Make the grid stretched horizontally
    ax.set_aspect('auto')


def show_value(ax, V, it):
    ax.set_title(f"Value Function v_{it}", fontsize=16)

    X, Y = np.meshgrid(np.arange(MAX_STORAGE+1), np.arange(MAX_STORAGE+1))

    # Horizontal wide 3D surface
    ax.plot_surface(X, Y, V.T, cmap="plasma", edgecolor="none", alpha=0.95)

    # Adjust the camera → more horizontal appearance
    ax.view_init(elev=25, azim=230)

    # Optional 2nd surface with lighter transparency (looks different)
    ax.plot_surface(X, Y, V.T, cmap='coolwarm', alpha=0.65)

    # Subtle horizontal grid lines
    ax.grid(True, linestyle='--', alpha=0.35)

    # Widening the figure instead of vertical tall
    plt.gcf().set_size_inches(26, 12)

    ax.set_xlabel("Location B (Horizontal Axis)", fontsize=12)
    ax.set_ylabel("Location A (Vertical Axis)", fontsize=12)
    ax.set_zlabel("State Value V(s)", fontsize=12)


## Running Complete Policy Iteration


In [29]:
def run_policy_iteration(fname="gbike_result.png"):
    V = np.zeros((MAX_STORAGE+1, MAX_STORAGE+1))
    PI = np.zeros((MAX_STORAGE+1, MAX_STORAGE+1), dtype=int)

    fig = plt.figure(figsize=(22,14))
    axes = [fig.add_subplot(2, 3, i+1) for i in range(5)]
    ax3d = fig.add_subplot(2,3,6, projection="3d")

    show_policy(axes[0], 0, PI)

    for k in range(1, 5):   # match Sutton & Barto figure count
        evaluate_policy(V, PI)
        stop = improve_policy(V, PI)
        show_policy(axes[k], k, PI)
        if stop: break

    show_value(ax3d, V, k)
    plt.savefig(fname)
    plt.close()
    print(f"Final output stored as {fname}")


# run full solver
run_policy_iteration()



>> Running Policy Evaluation...
   Policy evaluation converged.
>> Improving Policy...

>> Running Policy Evaluation...
   Policy evaluation converged.
>> Improving Policy...

>> Running Policy Evaluation...
   Policy evaluation converged.
>> Improving Policy...

>> Running Policy Evaluation...
   Policy evaluation converged.
>> Improving Policy...
Final output stored as gbike_result.png
